# Classification Main-Pied - REGION INTERET

## Apprentissage sur stimulation; Test sur imagination

Classification sur voxels 
Masque système moteur obtenus à partir de cartes ICA de NeuroVault (meta analyses) en selectionant mots clés: motor, premotor, sma, foot, hand 
exemple: http://nilearn.github.io/auto_examples/05_advanced/plot_ica_neurovault.html#sphx-glr-auto-examples-05-advanced-plot-ica-neurovault-py

Tous les sujets - Classifieur: GraphNet